In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 3.9 MB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
     |████████████████████████████████| 3.3 MB 48.0 MB/s 
     |████████████████████████████████| 61 kB 651 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 22.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=ec7d393151f141221d8efe61c7122d42af04ffd0873c67ead75212b06b91852b
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [ ]:
!pip install datasets

     |████████████████████████████████| 298 kB 4.0 MB/s 
     |████████████████████████████████| 132 kB 59.2 MB/s 
     |████████████████████████████████| 243 kB 58.1 MB/s 
     |████████████████████████████████| 1.1 MB 69.1 MB/s 
     |████████████████████████████████| 271 kB 73.5 MB/s 
     |████████████████████████████████| 192 kB 84.5 MB/s 
     |████████████████████████████████| 160 kB 85.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = 'drive/My Drive/6.864-project/code'
import os
os.chdir(root_path)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import scipy
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from collections import OrderedDict
from tqdm import tqdm
import torch
from datasets import Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling
from sentence_transformers import SentenceTransformer
from core_functions import evaluate_model

In [ ]:
device = "cuda:0"

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = AutoModel.from_pretrained('bert-base-uncased').to(device)

roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-base')
roberta_model = AutoModel.from_pretrained('roberta-base').to(device)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence_model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
train_data = pd.read_csv('../data/processed/training_data.csv')
val_data = pd.read_csv('../data/processed/val_data.csv')
test_data = pd.read_csv('../data/processed/test_data.csv')

In [ ]:
datasets = OrderedDict([('train', {'notes': train_data.iloc[:,:22].drop_duplicates().reset_index(drop=True),
                                   'notes_ratings': train_data}),
                        ('val', {'notes': val_data.iloc[:, :22].drop_duplicates().reset_index(drop=True),
                                 'notes_ratings': val_data}),
                        ('test', {'notes': test_data.iloc[:, :22].drop_duplicates().reset_index(drop=True),
                                  'notes_ratings': test_data})])


In [ ]:
# Transformer finetuning on Masked LM objective (based on Hugging Face implementation)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

device = "cuda:0"

train = list(datasets['train']['notes']['summary'])
val = list(datasets['val']['notes']['summary'])

dataset_dict = {"train": {"text": train}, "val": {"text": val}}
dataset = DatasetDict()
for k,v in dataset_dict.items():
    dataset[k] = Dataset.from_dict(v)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

model = AutoModelForMaskedLM.from_pretrained('roberta-base')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

training_args = TrainingArguments("roberta_base", evaluation_strategy='epoch', save_strategy='epoch', per_device_train_batch_size=8, per_device_eval_batch_size=8, num_train_epochs=10, save_total_limit=1, 
load_best_model_at_end=True, metric_for_best_model="eval_loss")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    data_collator=data_collator,
)

trainer.train()
trainer.save_model()


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

***** Running training *****
  Num examples = 8861
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 11080


Epoch,Training Loss,Validation Loss
1,1.750500,1.579043
2,1.564000,1.501098
3,1.495200,1.484237
4,1.395900,1.440905
5,1.342900,1.442873
6,1.262400,1.433101
7,1.214700,1.363221
8,1.163600,1.295218
9,1.122300,1.301417
10,1.071700,1.308551


***** Running Evaluation *****
  Num examples = 1108
  Batch size = 8
Saving model checkpoint to roberta_base/checkpoint-1108
Configuration saved in roberta_base/checkpoint-1108/config.json
Model weights saved in roberta_base/checkpoint-1108/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1108
  Batch size = 8
Saving model checkpoint to roberta_base/checkpoint-2216
Configuration saved in roberta_base/checkpoint-2216/config.json
Model weights saved in roberta_base/checkpoint-2216/pytorch_model.bin
Deleting older checkpoint [roberta_base/checkpoint-1108] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1108
  Batch size = 8
Saving model checkpoint to roberta_base/checkpoint-3324
Configuration saved in roberta_base/checkpoint-3324/config.json
Model weights saved in roberta_base/checkpoint-3324/pytorch_model.bin
Deleting older checkpoint [roberta_base/checkpoint-2216] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 

In [ ]:
device = "cuda:0"
model_path = "bert_base"
finetuned_model = AutoModelForMaskedLM.from_pretrained(model_path).to(device)

In [ ]:
device = "cuda:0"
model_path = "roberta_base"
finetuned_roberta_model = AutoModelForMaskedLM.from_pretrained(model_path).to(device)

In [ ]:
def get_final_cls_hidden_state(model, tokenizer, text):
    # final layer CLS token hidden state
    inputs = tokenizer(text, truncation=True, return_tensors="pt").to(device)
    outputs = model(**inputs, output_hidden_states=True)
    hidden_states = outputs.hidden_states
    return hidden_states[-1][:,0,:].detach().cpu().numpy().flatten()
  
def get_average_hidden_state(model, tokenizer, text):
  # average of second last layer hidden states
  inputs = tokenizer(text, truncation=True, return_tensors="pt").to(device)
  outputs = model(**inputs, output_hidden_states=True)
  hidden_states = outputs.hidden_states
  return torch.mean(hidden_states[-2][0], dim=0).detach().cpu().numpy().flatten()

# BERT sentence transformer embedding
def get_sentence_transformer_embedding(model, text):
  return model.encode(text)


In [ ]:
df_dict = {'bert variant': [], 'classifier': [], 'embedding': [], 'dataset': [], 'accuracy': [], 'f1': [], 'roc-auc': [], 'precision': [], 'recall': [], 'avg_precision': []}

In [ ]:
def create_embeddings(func, model, tokenizer):
  for name, data_dict in tqdm(datasets.items(), total=3):
    notes_df = data_dict['notes']
    notes_ratings_df = data_dict['notes_ratings']
    
    bert_embeddings = []
    for i in range(len(notes_df)):
      bert_embeddings.append(func(model, tokenizer, notes_df['summary'][i]))
    
    X_df = pd.DataFrame(np.array(bert_embeddings))
    X_df['noteId'] = notes_df['noteId']
    merged_X_df = X_df.merge(notes_ratings_df[['noteId', 'output', 'scaling']],
                        how='inner', on='noteId')
    X = merged_X_df.drop(['noteId', 'output', 'scaling'], axis=1)
    datasets[name]['X'] = X
    datasets[name]['Y'] = merged_X_df['output']
    datasets[name]['scaling'] = merged_X_df['scaling']
    datasets[name]['noteId'] = merged_X_df['noteId']
  
  return datasets
    

In [ ]:
def create_sentence_embeddings():
  for name, data_dict in tqdm(datasets.items(), total=3):
    notes_df = data_dict['notes']
    notes_ratings_df = data_dict['notes_ratings']
    
    bert_embeddings = []
    for i in range(len(notes_df)):
      bert_embeddings.append(get_sentence_transformer_embedding(sentence_model, notes_df['summary'][i]))
    
    X_df = pd.DataFrame(np.array(bert_embeddings))
    X_df['noteId'] = notes_df['noteId']
    merged_X_df = X_df.merge(notes_ratings_df[['noteId', 'output', 'scaling']],
                        how='inner', on='noteId')
    X = merged_X_df.drop(['noteId', 'output', 'scaling'], axis=1)
    datasets[name]['X'] = X
    datasets[name]['Y'] = merged_X_df['output']
    datasets[name]['scaling'] = merged_X_df['scaling']
    datasets[name]['noteId'] = merged_X_df['noteId']
    return datasets

In [ ]:
def classify(datasets, model, embedding):
  logistic = LogisticRegression(random_state=7)
  gradient = GradientBoostingClassifier(random_state=7, verbose=1)
  logistic.fit(datasets['train']['X'], datasets['train']['Y'],
            sample_weight=datasets['train']['scaling'])
  gradient.fit(datasets['train']['X'], datasets['train']['Y'],
            sample_weight=datasets['train']['scaling'])

  classifiers = {'logistic': logistic, 'gradient': gradient}

  for clf in classifiers:
    classifier = classifiers[clf]
    for name, data_dict in datasets.items():
      print(name)
      predictions = classifier.predict(data_dict['X'])
      p_hat = classifier.predict_proba(data_dict['X'])[:, 1]
      observations = data_dict['Y']
      scaling = data_dict['scaling']
      performance_dict = evaluate_model(predictions, p_hat, observations,
                                        scaling, name)
      data_dict['performance'] = performance_dict
      df_dict['bert variant'].append(model)
      df_dict['classifier'].append(clf)
      df_dict['embedding'].append(embedding)
      df_dict['dataset'].append(name)
      df_dict['accuracy'].append(performance_dict['accuracy'])
      df_dict['f1'].append(performance_dict['f1'])
      df_dict['roc-auc'].append(performance_dict['roc-auc'])
      df_dict['precision'].append(performance_dict['precision'])
      df_dict['recall'].append(performance_dict['recall'])
      df_dict['avg_precision'].append(performance_dict['avg_precision'])

In [ ]:
new_datasets = create_embeddings(get_final_cls_hidden_state, bert_model, bert_tokenizer)
classify(new_datasets, 'bert', 'cls')

100%|██████████| 3/3 [02:36<00:00, 52.32s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


      Iter       Train Loss   Remaining Time 
         1           1.2922           16.37m
         2           1.2826           16.22m
         3           1.2746           16.12m
         4           1.2671           15.99m
         5           1.2607           15.81m
         6           1.2551           15.61m
         7           1.2498           15.41m
         8           1.2454           15.27m
         9           1.2415           15.10m
        10           1.2377           14.93m
        20           1.2137           13.26m
        30           1.1981           11.65m
        40           1.1871           10.05m
        50           1.1781            8.41m
        60           1.1694            6.75m
        70           1.1612            5.08m
        80           1.1535            3.39m
        90           1.1470            1.70m
       100           1.1392            0.00s
train
val
test
train
val
test


In [ ]:
new_datasets = create_embeddings(get_final_cls_hidden_state, roberta_model, roberta_tokenizer)
classify(new_datasets, 'roberta', 'cls')

100%|██████████| 3/3 [02:40<00:00, 53.50s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


      Iter       Train Loss   Remaining Time 
         1           1.2925           16.38m
         2           1.2832           16.20m
         3           1.2749           16.06m
         4           1.2682           15.89m
         5           1.2621           15.72m
         6           1.2568           15.57m
         7           1.2518           15.40m
         8           1.2474           15.23m
         9           1.2434           15.08m
        10           1.2398           14.91m
        20           1.2153           13.26m
        30           1.1988           11.61m
        40           1.1859           10.00m
        50           1.1762            8.38m
        60           1.1668            6.72m
        70           1.1588            5.06m
        80           1.1506            3.38m
        90           1.1439            1.70m
       100           1.1376            0.00s
train
val
test
train
val
test


In [ ]:
new_datasets = create_embeddings(get_average_hidden_state, bert_model, bert_tokenizer)
classify(new_datasets, 'bert', 'second last')

100%|██████████| 3/3 [02:36<00:00, 52.02s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


      Iter       Train Loss   Remaining Time 
         1           1.2901           16.36m
         2           1.2792           16.20m
         3           1.2702           16.03m
         4           1.2626           15.87m
         5           1.2560           15.69m
         6           1.2503           15.52m
         7           1.2454           15.36m
         8           1.2414           15.22m
         9           1.2377           15.07m
        10           1.2342           14.92m
        20           1.2106           13.28m
        30           1.1958           11.65m
        40           1.1843           10.04m
        50           1.1747            8.41m
        60           1.1665            6.76m
        70           1.1589            5.09m
        80           1.1531            3.41m
        90           1.1459            1.70m
       100           1.1393            0.00s
train
val
test
train
val
test


In [ ]:
new_datasets = create_embeddings(get_average_hidden_state, roberta_model, roberta_tokenizer)
classify(new_datasets, 'roberta', 'second last')

100%|██████████| 3/3 [02:40<00:00, 53.49s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


      Iter       Train Loss   Remaining Time 
         1           1.2913           16.53m
         2           1.2813           16.42m
         3           1.2726           16.26m
         4           1.2655           16.10m
         5           1.2592           15.91m
         6           1.2536           15.74m
         7           1.2489           15.58m
         8           1.2446           15.39m
         9           1.2406           15.22m
        10           1.2371           15.05m
        20           1.2136           13.38m
        30           1.1979           11.72m
        40           1.1857           10.08m
        50           1.1752            8.42m
        60           1.1662            6.76m
        70           1.1582            5.09m
        80           1.1500            3.40m
        90           1.1419            1.70m
       100           1.1352            0.00s
train
val
test
train
val
test


In [ ]:
new_datasets = create_embeddings(get_final_cls_hidden_state, finetuned_model, bert_tokenizer)
classify(new_datasets, 'finetuned', 'cls')

100%|██████████| 3/3 [02:44<00:00, 54.98s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


      Iter       Train Loss   Remaining Time 
         1           1.2914           16.39m
         2           1.2813           16.16m
         3           1.2726           16.00m
         4           1.2652           15.88m
         5           1.2584           15.67m
         6           1.2527           15.51m
         7           1.2472           15.35m
         8           1.2427           15.20m
         9           1.2386           15.04m
        10           1.2346           14.87m
        20           1.2091           13.25m
        30           1.1926           11.62m
        40           1.1805           10.01m
        50           1.1701            8.37m
        60           1.1615            6.72m
        70           1.1542            5.06m
        80           1.1463            3.38m
        90           1.1404            1.70m
       100           1.1341            0.00s
train
val
test
train
val
test


In [ ]:
new_datasets = create_embeddings(get_average_hidden_state, finetuned_model, bert_tokenizer)
classify(new_datasets, 'finetuned', 'second last')

100%|██████████| 3/3 [02:45<00:00, 55.29s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


      Iter       Train Loss   Remaining Time 
         1           1.2908           16.57m
         2           1.2804           16.42m
         3           1.2716           16.25m
         4           1.2640           16.04m
         5           1.2576           15.88m
         6           1.2516           15.70m
         7           1.2465           15.51m
         8           1.2420           15.34m
         9           1.2379           15.18m
        10           1.2344           15.02m
        20           1.2094           13.35m
        30           1.1932           11.69m
        40           1.1811           10.06m
        50           1.1705            8.41m
        60           1.1608            6.74m
        70           1.1526            5.07m
        80           1.1451            3.39m
        90           1.1377            1.70m
       100           1.1311            0.00s
train
val
test
train
val
test


In [ ]:
new_datasets = create_sentence_embeddings()
classify(new_datasets, 'sentence', 'sentence')

  0%|          | 0/3 [02:12<?, ?it/s]


      Iter       Train Loss   Remaining Time 
         1           1.2968           16.18m
         2           1.2912           16.09m
         3           1.2859           15.88m
         4           1.2813           15.75m
         5           1.2773           15.60m
         6           1.2733           15.44m
         7           1.2698           15.28m
         8           1.2662           15.11m
         9           1.2631           14.94m
        10           1.2598           14.77m
        20           1.2350           13.20m
        30           1.2169           11.59m
        40           1.2028            9.99m
        50           1.1909            8.36m
        60           1.1813            6.72m
        70           1.1720            5.05m
        80           1.1623            3.37m
        90           1.1557            1.69m
       100           1.1482            0.00s
train
val
test
train
val
test


In [ ]:
df = pd.DataFrame(df_dict)

In [ ]:
df.to_csv("results.csv")

In [ ]:
df = pd.read_csv('results.csv')

In [ ]:
df_dict = df.to_dict('list')

In [ ]:
del df_dict['Unnamed: 0']

In [ ]:
new_datasets = create_embeddings(get_final_cls_hidden_state, finetuned_roberta_model, roberta_tokenizer)
classify(new_datasets, 'finetuned roberta', 'cls')

new_datasets = create_embeddings(get_average_hidden_state, finetuned_roberta_model, roberta_tokenizer)
classify(new_datasets, 'finetuned roberta', 'second last')

100%|██████████| 3/3 [01:54<00:00, 38.32s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


      Iter       Train Loss   Remaining Time 
         1           1.2929           14.61m
         2           1.2834           14.49m
         3           1.2753           14.41m
         4           1.2682           14.25m
         5           1.2619           14.12m
         6           1.2565           13.95m
         7           1.2515           13.81m
         8           1.2468           13.65m
         9           1.2427           13.52m
        10           1.2388           13.37m
        20           1.2132           11.93m
        30           1.1962           10.49m
        40           1.1831            9.03m
        50           1.1721            7.54m
        60           1.1636            6.07m
        70           1.1555            4.56m
        80           1.1488            3.06m
        90           1.1422            1.53m
       100           1.1346            0.00s
train
val
test
train
val
test


100%|██████████| 3/3 [01:53<00:00, 37.94s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


      Iter       Train Loss   Remaining Time 
         1           1.2911           15.16m
         2           1.2808           14.91m
         3           1.2722           14.73m
         4           1.2644           14.54m
         5           1.2582           14.38m
         6           1.2526           14.22m
         7           1.2478           14.04m
         8           1.2434           13.90m
         9           1.2396           13.75m
        10           1.2361           13.58m
        20           1.2118           12.06m
        30           1.1961           10.55m
        40           1.1828            9.07m
        50           1.1714            7.57m
        60           1.1624            6.08m
        70           1.1546            4.57m
        80           1.1476            3.06m
        90           1.1408            1.53m
       100           1.1342            0.00s
train
val
test
train
val
test


In [ ]:
df = pd.DataFrame(df_dict)
df.to_csv("results.csv")

In [ ]:
df

,bert variant,classifier,embedding,dataset,accuracy,f1,roc-auc,precision,recall,avg_precision
0,bert,logistic,cls,train,0.689790,0.783942,0.707605,0.709760,0.875441,0.799461
1,bert,logistic,cls,val,0.672613,0.770447,0.658318,0.693325,0.866875,0.735608
2,bert,logistic,cls,test,0.657862,0.761003,0.657204,0.689758,0.848661,0.756457
3,bert,gradient,cls,train,0.709842,0.799827,0.752312,0.718608,0.901745,0.836234
4,bert,gradient,cls,val,0.672353,0.774685,0.658911,0.686573,0.888742,0.745322
5,bert,gradient,cls,test,0.668170,0.768932,0.661589,0.695169,0.860206,0.762770
6,roberta,logistic,cls,train,0.683080,0.782483,0.687608,0.700172,0.886724,0.782810
7,roberta,logistic,cls,val,0.670415,0.772929,0.658864,0.686006,0.885077,0.742983
8,roberta,logistic,cls,test,0.671317,0.770691,0.666968,0.697825,0.860548,0.765696
9,roberta,gradient,cls,train,0.714983,0.802945,0.754759,0.722667,0.903287,0.839031


In [ ]:
train = df[df['dataset'] == 'train']
train[train['accuracy'] == max(train['accuracy'])]

,bert variant,classifier,embedding,dataset,accuracy,f1,roc-auc,precision,recall,avg_precision
21,roberta,gradient,second last,train,0.716525,0.805994,0.750747,0.719587,0.915984,0.830979


In [ ]:
val = df[df['dataset'] == 'val']
val[val['accuracy'] == max(val['accuracy'])]

,bert variant,classifier,embedding,dataset,accuracy,f1,roc-auc,precision,recall,avg_precision
19,roberta,logistic,second last,val,0.688846,0.786292,0.655706,0.696193,0.903178,0.736669


In [ ]:
val[val['roc-auc'] == max(val['roc-auc'])]

,bert variant,classifier,embedding,dataset,accuracy,f1,roc-auc,precision,recall,avg_precision
28,finetuned,gradient,cls,val,0.674232,0.774859,0.667133,0.689385,0.884528,0.750729


In [ ]:
val[val['f1'] == max(val['f1'])]

,bert variant,classifier,embedding,dataset,accuracy,f1,roc-auc,precision,recall,avg_precision
19,roberta,logistic,second last,val,0.688846,0.786292,0.655706,0.696193,0.903178,0.736669


In [ ]:
test = df[df['dataset'] == 'test']
test[test['accuracy'] == max(test['accuracy'])]

,bert variant,classifier,embedding,dataset,accuracy,f1,roc-auc,precision,recall,avg_precision
50,finetuned roberta,logistic,second last,test,0.677794,0.77321,0.665542,0.705192,0.855751,0.763419


In [ ]:
test[test['roc-auc'] == max(test['roc-auc'])]

,bert variant,classifier,embedding,dataset,accuracy,f1,roc-auc,precision,recall,avg_precision
44,finetuned roberta,logistic,cls,test,0.677051,0.774265,0.668982,0.702139,0.862906,0.768513


In [ ]:
test[test['f1'] == max(test['f1'])]

,bert variant,classifier,embedding,dataset,accuracy,f1,roc-auc,precision,recall,avg_precision
44,finetuned roberta,logistic,cls,test,0.677051,0.774265,0.668982,0.702139,0.862906,0.768513
